<a href="https://colab.research.google.com/github/JuanBustamante107517/Lab_08_Bustamante_G/blob/main/Lab_04_Bustamante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
breast_cancer_wisconsin_original = fetch_ucirepo(id=15)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_original.data.features
y = breast_cancer_wisconsin_original.data.targets

# metadata
print(breast_cancer_wisconsin_original.metadata)

# variable information
print(breast_cancer_wisconsin_original.variables)


{'uci_id': 15, 'name': 'Breast Cancer Wisconsin (Original)', 'repository_url': 'https://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original', 'data_url': 'https://archive.ics.uci.edu/static/public/15/data.csv', 'abstract': 'Original Wisconsin Breast Cancer Database', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 699, 'num_features': 9, 'feature_types': ['Integer'], 'demographics': [], 'target_col': ['Class'], 'index_col': ['Sample_code_number'], 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1990, 'last_updated': 'Sun Mar 10 2024', 'dataset_doi': '10.24432/C5HP4Z', 'creators': ['WIlliam Wolberg'], 'intro_paper': None, 'additional_info': {'summary': "Samples arrive periodically as Dr. Wolberg reports his clinical cases. The database therefore reflects this chronological grouping of the data. This grouping information appears immediately below, having been removed fro

In [3]:
import pandas as pd
import numpy as np

### a. Por medio de la librería ‘Pandas’, lea la base de datos, asigne nombres para las columnas,realice una imputación para la variable ‘Bare Nuclei’ por medio de la mediana y cambie las categorías para la variable ‘Class’ de 2 o 4 por 0 o 1.

In [4]:
# Ya tienes X (features) y y (target) cargados desde ucimlrepo
# Verificamos las columnas:
print("Columnas disponibles en X:")
print(X.columns)

Columnas disponibles en X:
Index(['Clump_thickness', 'Uniformity_of_cell_size',
       'Uniformity_of_cell_shape', 'Marginal_adhesion',
       'Single_epithelial_cell_size', 'Bare_nuclei', 'Bland_chromatin',
       'Normal_nucleoli', 'Mitoses'],
      dtype='object')


In [6]:
# Aseguramos que 'Bare_Nuclei' esté como numérico
X['Bare_nuclei'] = pd.to_numeric(X['Bare_nuclei'], errors='coerce')



<ipython-input-6-4c78f68fbf6c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Bare_nuclei'] = pd.to_numeric(X['Bare_nuclei'], errors='coerce')


In [7]:
# Imputación de la mediana para 'Bare_Nuclei'
X['Bare_nuclei'].fillna(X['Bare_nuclei'].median(), inplace=True)

<ipython-input-7-25c2a6e44435>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['Bare_nuclei'].fillna(X['Bare_nuclei'].median(), inplace=True)
<ipython-input-7-25c2a6e44435>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Bare_nuclei'].fillna(X['Bare_nuclei'].median(), inplace=True)


In [8]:
# Verificamos los valores únicos antes de cambiar en y
print("Valores originales en y:")
print(y['Class'].value_counts())

Valores originales en y:
Class
2    458
4    241
Name: count, dtype: int64


In [9]:
# Reemplazar 2 → 0 y 4 → 1
y['Class'] = y['Class'].replace({2: 0, 4: 1})

<ipython-input-9-12059bf629f1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['Class'] = y['Class'].replace({2: 0, 4: 1})


In [10]:
# Verificar los cambios
print("Valores nuevos en y:")
print(y['Class'].value_counts())

Valores nuevos en y:
Class
0    458
1    241
Name: count, dtype: int64


In [11]:
# Mostrar primeras filas para confirmar
print(X.head())
print(y.head())

   Clump_thickness  Uniformity_of_cell_size  Uniformity_of_cell_shape  \
0                5                        1                         1   
1                5                        4                         4   
2                3                        1                         1   
3                6                        8                         8   
4                4                        1                         1   

   Marginal_adhesion  Single_epithelial_cell_size  Bare_nuclei  \
0                  1                            2          1.0   
1                  5                            7         10.0   
2                  1                            2          2.0   
3                  1                            3          4.0   
4                  3                            2          1.0   

   Bland_chromatin  Normal_nucleoli  Mitoses  
0                3                1        1  
1                3                2        1  
2                3     

### b. Realice una detección de valores atípicos univariados por medio del método del rango intercuartílico con 3 de longitud a la derecha y 3 a la izquierda, y elimínelos. Además, realiceuna detección de valores atípicos multivariados por medio de las distancias de Mahalanobis y elimine aquellos valores que superen el valor de 30.

In [12]:
from scipy.spatial.distance import mahalanobis
import numpy as np
import pandas as pd

In [13]:
# --- COPIA DEL DATASET ORIGINAL ---
X_clean = X.copy()

In [14]:
# --- DETECCIÓN UNIVARIADA CON IQR (rango intercuartílico) ---
Q1 = X_clean.quantile(0.25)
Q3 = X_clean.quantile(0.75)
IQR = Q3 - Q1

In [16]:
# Eliminamos filas que tengan outliers en alguna columna (usando margen de 3 * IQR)
condition = ~((X_clean < (Q1 - 3 * IQR)) | (X_clean > (Q3 + 3 * IQR))).any(axis=1)
X_iqr = X_clean[condition]
y_iqr = y[condition]  # también filtramos y para que coincida
print(f"Filas después de eliminar atípicos univariados: {X_iqr.shape[0]}")

Filas después de eliminar atípicos univariados: 579


In [18]:
from scipy.spatial.distance import mahalanobis
import numpy as np

# --- DETECCIÓN MULTIVARIADA CON MAHALANOBIS usando pseudo-inversa ---
mean_vec = X_iqr.mean().values
cov_matrix = np.cov(X_iqr.values, rowvar=False)
inv_cov_matrix = np.linalg.pinv(cov_matrix)  # ← cambio aquí

mahal_distances = X_iqr.apply(lambda row: mahalanobis(row, mean_vec, inv_cov_matrix), axis=1)


In [19]:
# Filtro por umbral
mask_mahal = mahal_distances <= 30
X_final = X_iqr[mask_mahal]
y_final = y_iqr[mask_mahal]

print(f"Filas después de limpieza multivariada (Mahalanobis): {X_final.shape[0]}")

Filas después de limpieza multivariada (Mahalanobis): 579


### c. Realice un cálculo de los principales estadísticos descriptivos para las variables predictoras y cree dos conjuntos de datos: uno en donde se aplique la transformación Z-score y otro en donde se aplique la normalización mín-máx, sobre todas las variables predictoras.

In [20]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [21]:
# Asegúrate de que todas las columnas sean numéricas
X_numeric = X.apply(pd.to_numeric, errors='coerce')

In [22]:
# ⚠️ Validamos si hay valores nulos antes de transformar
print("¿Hay valores nulos?:")
print(X_numeric.isnull().sum())

¿Hay valores nulos?:
Clump_thickness                0
Uniformity_of_cell_size        0
Uniformity_of_cell_shape       0
Marginal_adhesion              0
Single_epithelial_cell_size    0
Bare_nuclei                    0
Bland_chromatin                0
Normal_nucleoli                0
Mitoses                        0
dtype: int64


In [23]:
# Si los hay, imputamos con la media (podrías usar median si prefieres)
X_numeric.fillna(X_numeric.mean(), inplace=True)

In [24]:
# 🔹 Transformación Z-score
scaler_z = StandardScaler()
X_zscore = pd.DataFrame(scaler_z.fit_transform(X_numeric), columns=X.columns)

In [25]:
# 🔹 Normalización Min-Máx
scaler_minmax = MinMaxScaler()
X_minmax = pd.DataFrame(scaler_minmax.fit_transform(X_numeric), columns=X.columns)

In [26]:
# 🔍 Mostramos resumen
print("\n📌 Primeras filas del conjunto Z-score:")
print(X_zscore.head())

print("\n📌 Primeras filas del conjunto Min-Máx:")
print(X_minmax.head())


📌 Primeras filas del conjunto Z-score:
   Clump_thickness  Uniformity_of_cell_size  Uniformity_of_cell_shape  \
0         0.206936                -0.699995                 -0.743299   
1         0.206936                 0.283845                  0.266875   
2        -0.503866                -0.699995                 -0.743299   
3         0.562336                 1.595632                  1.613773   
4        -0.148465                -0.699995                 -0.743299   

   Marginal_adhesion  Single_epithelial_cell_size  Bare_nuclei  \
0          -0.633247                    -0.549561    -0.686979   
1           0.768621                     1.710106     1.799664   
2          -0.633247                    -0.549561    -0.410685   
3          -0.633247                    -0.097628     0.141902   
4           0.067687                    -0.549561    -0.686979   

   Bland_chromatin  Normal_nucleoli   Mitoses  
0        -0.179662        -0.611825 -0.343912  
1        -0.179662        -0